In [1]:
''' Do not change this cell '''
#Functions

def check_gpu():
  if tf.test.gpu_device_name() != '':
    print("GPU sucessfully connected")
  else: print("PLease connect GPU")



# **#Task 1**#


In [2]:
#import all the required packages and  libraries for FineTuning LLM
!pip install transformers datasets accelerate bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import tensorflow as tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
''' Do not change this cell '''
check_gpu()


GPU sucessfully connected


#**Copy this task at end of task #2 notebook**#

In [27]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ColabToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ColabTo

In [11]:
# !pip install -q datasets
# from datasets import load_dataset

splits = {'train': 'train.csv', 'validation': 'validation.csv', 'test': 'test.csv'}
df = load_dataset('jizzu/llama2_indian_law_v2')
dfTrain = df['train']
dfVal = df['validation']
dfTest = df['test']

In [23]:
print(dfTrain[0])
print(type(dfTrain[0]['text']))

{'text': "###Human:\nWhat is the difference between a petition and a plaint in Indian law?\n###Assistant:\nA petition is a formal request submitted to a courttribunalor authority to seek a specific remedy or relief. It is commonly used for various purposessuch as filing a writ petition in the High Court or submitting a petition for divorce. On the other handa plaint is a formal written statement of a plaintiff's claim in a civil lawsuit. The key difference is that a petition is more versatile and can be used for various legal matterswhile a plaint is specific to civil cases."}
<class 'str'>


In [5]:
def preprocessDataset(df,formattedList):

    for rowData in df.iterrows():
        question = rowData[1][0].split("###Human:")[1].split("###Assistant:")[0].strip()
        answer = rowData[1][0].split("###Assistant:")[1].strip()
        formattedList.append({"<s>[INST] {"+question+"} [/INST]{"+answer+"}</s>"});


In [24]:
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ' '.join(reformatted_segments)}

In [25]:
formattedTestList = dfTest.map(transform_conversation)
formattedTrainList = dfTrain.map(transform_conversation)
formattedValList = dfVal.map(transform_conversation)

Map:   0%|          | 0/276 [00:00<?, ? examples/s]

Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

Map:   0%|          | 0/455 [00:00<?, ? examples/s]

In [28]:
formattedTestList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Test")
formattedTrainList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Train")
formattedValList.push_to_hub("llama2_indian_law_v2_formatted_data",split="Validation")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/266 [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/374 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/QuantTitan/llama2_indian_law_v2_formatted_data/commit/f999cc3476e841912811a2d4194a73ed5f0447e6', commit_message='Upload dataset', commit_description='', oid='f999cc3476e841912811a2d4194a73ed5f0447e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/QuantTitan/llama2_indian_law_v2_formatted_data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='QuantTitan/llama2_indian_law_v2_formatted_data'), pr_revision=None, pr_num=None)